In [1]:
import pandas as pd
from matplotlib import pyplot as plt
import datetime
%matplotlib inline
import seaborn as sns
sns.set()
from matplotlib.backends.backend_pdf import PdfPages
#from mpl_toolkits.basemap import Basemap
import matplotlib.cm as cm
import matplotlib.colors as col
import matplotlib as mpl
import numpy as np #import for transposing
import math
import pylab as pl
import math
import import_ipynb
from sklearn.metrics import mean_squared_error,mean_absolute_error
from function_log import add_tdate,get_week,density_interp,extract_date,qtr_comp,IC_run_sim,new_compare,stoch_model,run_sim

from scipy.optimize import minimize

importing Jupyter notebook from function_log.ipynb


In [2]:
def find(condition):
    res, = np.nonzero(np.ravel(condition))
    return res

In [3]:
new_census = pd.read_csv('new_data.csv')
new_census['Date'] = pd.to_datetime(new_census['Date'])
new_census['ObsYear'] = new_census.Date.apply(lambda x: x.year)
new_census['ObsMonth'] = new_census.Date.apply(lambda x:x.month)

new_census.T = pd.to_numeric(new_census['T'],errors='coerce')
new_census.J = pd.to_numeric(new_census.J)
new_census.A = pd.to_numeric(new_census.A)
new_census.S = pd.to_numeric(new_census.S)

In [4]:
t_types = pd.read_csv('troop_types.csv')

rural_troops = list(t_types.loc[t_types['Type Code'] == 0]['Troop'])
rural = new_census.loc[new_census.Troop.isin(rural_troops)]

urb_troops = list(t_types.loc[t_types['Type Code'] == 1]['Troop'])
urb_troops.append('KUB')
urban = new_census.loc[new_census.Troop.isin(urb_troops)]

In [5]:
from __future__ import print_function
import sys
import threading
from time import sleep
try:
    import thread
except ImportError:
    import _thread as thread

def quit_function():
        # print to stderr, unbuffered in Python 2.
        print('Parameter set rejected', file=sys.stderr)
        sys.stderr.flush() # Python 3 stderr is likely buffered.
        thread.interrupt_main() # raises KeyboardInterrupt
        

def exit_after(s):
    '''
    use as decorator to exit process if 
    function takes longer than s seconds
    '''
    def outer(fn):
        def inner(*args, **kwargs):
            timer = threading.Timer(s, quit_function, args=[])
            timer.start()
            try:
                result = fn(*args, **kwargs)
            finally:
                timer.cancel()
            return result
        return inner
    return outer

##Initialize timeout function
@exit_after(120)
def min_func(paramset):
    
    with open('free_paramset_rural.txt', 'a') as param:
        param.write('{params}\n'.format(params=paramset))

        
    try:    
        ##Extract median NRMSE value from model run
        res_df = new_compare(rural,iters=25,
                             fus_fis=paramset[0],
                             no_ev=paramset[1],
                             rt_chg=paramset[2],
                             b_j=paramset[3],
                             d_j=paramset[4],
                             d_s=paramset[5],
                             d_a=paramset[6],
                             ae_j=paramset[7],
                             ae_s=paramset[8],
                             ae_a=paramset[9],
                             t_p=paramset[10],
                             por=paramset[11],
                             fsr=paramset[12],
                             thresh=paramset[13],
                             fis_thresh=paramset[14],
                             lit_size=paramset[15],
                             dfe='off'
                            )
        smape = res_df.loc[res_df.sMAPE<=100]
        smape = smape.loc[:,'sMAPE'].dropna()

        total = len(smape)
        good = len(smape.loc[smape <= 15])
        perc = 1-(good/total)
        perc*=100
        
        print('Percent of High Error Runs: {}\n'.format(perc))
        with open('free_percenterr_rural.txt', 'a') as percenterr:
            percenterr.write('{percents}\n'.format(percents=perc))
        
        
        return perc
    
    except:
        with open('rejected_rur_params.txt', 'a') as param:
            param.write('{time}: {params}\n'.format(time = str(datetime.datetime.now()),params=paramset))
        try:    
            with open('free_percenterr_rural.txt','r') as f:
                last_perc = float(f.readlines()[-1][:-2])

            perc = last_perc + 5
            
        except:
            perc = 1
        
        print('Parameter set rejected. Returned percent: {}'.format(perc))
        
        
        return perc

In [ ]:
##USE PYSWARM HERE
from pyswarm import pso
import warnings
warnings.filterwarnings('ignore')
#x0 = [0.4,0.8,0.1,0.299,0.345,0.216,0.143,0.5,0.5,0.5,0.3,0.2,0.4,4,45,4]

lb = [0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,4,38,3]
ub = [1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,6,68,5]

xopt,fopt = pso(min_func,lb,ub,maxiter=25,minstep=0.001,debug=True,minfunc=0.001)

No constraints given.


Parameter set rejected


2021-03-22 17:03:40.088725 calculated new_compare
Percent of High Error Runs: 95.61815336463224

2021-03-22 17:04:19.890284 calculated new_compare
Percent of High Error Runs: 95.74144486692016

2021-03-22 17:05:01.720404 calculated new_compare
Percent of High Error Runs: 80.0

2021-03-22 17:05:43.045877 calculated new_compare
Percent of High Error Runs: 72.6890756302521

2021-03-22 17:06:22.401596 calculated new_compare
Percent of High Error Runs: 72.80071813285458

2021-03-22 17:07:02.396628 calculated new_compare
Percent of High Error Runs: 95.22799575821846



Parameter set rejected


Parameter set rejected. Returned percent: 100.2279957582184
Parameter set rejected. Returned percent: 100.2279957582184
2021-03-22 17:12:51.306437 calculated new_compare
Percent of High Error Runs: 96.51623119556612

Parameter set rejected. Returned percent: 101.5162311955661


...

Parameter set rejected. Returned percent: 101.1194029850746
2021-03-22 19:00:17.323141 calculated new_compare
Percent of High Error Runs: 94.98525073746312

2021-03-22 19:01:02.073483 calculated new_compare
Percent of High Error Runs: 92.25847728726806



Parameter set rejected


Parameter set rejected. Returned percent: 97.258477287268


Parameter set rejected


Parameter set rejected. Returned percent: 97.258477287268


In [ ]:
xopt
with open('free_optimiz_rural1.txt', 'a') as optimiz:
        optimiz.write('{params}\n'.format(params=xopt))

In [ ]:
fopt